In [1]:
import numpy as np 
import pandas as pd 

In [ ]:
#df=pd.read_csv('/home/safiyyah/Documents/Courses/project3004/Oto-InnerEar/organized.csv')
#df=pd.read_csv('organized.csv', encoding = "ISO-8859-1")

The database contains TEOAE signals collected from 54 individuals or subjects.
For each subject there are 2 recording sessions - 0 and 1 - separated by 1 week, atleast.
In each session, recording is done for both right and left ears.
For each ear, recording is done using 2 microphones (called as 2 buffers A and B)
The raw dataset has a total of 50986 TEOAE signals recorded from 54 subjects - (data matrix size:50986 x 665)

In [ ]:
df.head()
df.shape[1]

In [ ]:
df.shape

In each row: columns with labels --> data_0 to data_659 contain one TEOAE response.  The no. of TEOAE responses for each condition (e.g., subject 0, session 0, ear = left, buffer =B) varies from 27 to 300.

In [2]:
#import plotly
#import plotly.graph_objs as go
import matplotlib.pyplot as plt

#plotly.offline.init_notebook_mode(connected=True)

The last 10 TEOAE signals are observed to be very similar (steady) for a person,
yet these signals are very different for different individuals --> i.e., intraclass variability is low, while interclass variability is high.

This could mean that different subject's TEOAE signals contain different frequency components.

So, we extract frequency information from the signals using Fast Fourier Transform (fft), power spectral density (psd) and autocorrelation --> these frequency related information is used as feature input to the machine learning algorithm.


In [ ]:
import pandas
import numpy
#df.groupby(['subject', 'session', 'ear', 'buffer'],as_index=False)['data_0',''].agg(lambda x:list(x))

#dfu0=df.groupby(['subject', 'session', 'ear', 'buffer']).get_group((0,0,'right','A')).iloc[-10:]

def subject0(subjectno, sessionno, earname, bufferid):
    dfu0=df.groupby(['subject', 'session', 'ear', 'buffer']).get_group((subjectno, sessionno, earname, bufferid)).iloc[-10:]
    return dfu0

x=subject0(34,1,'left','B')
x['subject'].iloc[0]

In [ ]:
# This extracts 80 responses for each of the 54 subjects: 
# for each subject: a total of 80 last 10 responses corresponding to the 8 different recording conditions
#the 8 conditions are:
# for subject = i --> session = 0/1 , ear = left/right, buffer = A/B ( 2 sessions * 2 ears * 2 buffers)

from sklearn import preprocessing
import pandas
import numpy

allInputTEOAEsignalsAllSubjects = pd.DataFrame([])
allOutputLabelsAllSubjects = pd.DataFrame([])

#temp=pd.DataFrame([])
for a in range(54):
    if a==48:
        print('passing on')
        continue
    #for each subject a 
    print(a)
    df1 = pd.DataFrame([])
    #a=list(range(0,1))
    b=[0,1]              # sessions 0 and 1
    c=['left','right']   # ears 
    d=['A','B']          # buffers
    
    allFeatures =  pd.DataFrame([])
    allInputFeatures =  pd.DataFrame([])
    outputLabel =  pd.DataFrame([])
    fftValues =  pd.DataFrame([])
    psdValues =  pd.DataFrame([])
    autocorrValues = pd.DataFrame([])
        
    for q,r,s in [(q,r,s) for q in b for r in c for s in d]:
        
        # df1 contains all the 80 TEOAE signals of subject a
        df1 = df1.append(subject0(a,q,r,s))
        allInputFeatures = df1.iloc[:,5:]
    
    allInputTEOAEsignalsAllSubjects = pd.concat([allInputTEOAEsignalsAllSubjects.reset_index(drop=True),
                                                 allInputFeatures.reset_index(drop=True)], axis=0)
    # the following gets you the class label <= subject no.[0,1,2,...,53]
    outputLabel = df1.iloc[:,0] 
    #allOutputLabelsAllSubjects = allOutputLabelsAllSubjects.append(outputLabel)
    #append didn't work for output so using pd.cat instead
    allOutputLabelsAllSubjects = pd.concat([allOutputLabelsAllSubjects.reset_index(drop=True),
                                            outputLabel.reset_index(drop=True)], axis=0)
    #print(allOutputLabelsAllSubjects.shape)


In [ ]:
print(allInputTEOAEsignalsAllSubjects.shape)
print(allOutputLabelsAllSubjects.shape)
X = allInputTEOAEsignalsAllSubjects
y = allOutputLabelsAllSubjects

In [ ]:
X=X.iloc[:3760]
y=y.iloc[:3760]

In [ ]:
# save extracted dataframe locally
xsave = X
ysave = y
xsave.to_pickle('dataX.pkl')
ysave.to_pickle('labelY.pkl')

In [ ]:
#X = allInputTEOAEsignalsAllSubjects.iloc[1101:1111,:]
#y = allOutputLabelsAllSubjects.iloc[1101:1111]
#print(X.shape)
#print(y.shape)

http://ataspinar.com/2018/04/04/machine-learning-with-signal-processing-techniques/

In [ ]:
#Load from pkl files
X = pd.read_pickle('dataX.pkl')
y = pd.read_pickle('labelY.pkl')

In [ ]:
#X=allInputFeatsAllSubjects.iloc[:,130:256]
#y=allOutputLabelsAllSubjects
#y=y.astype(int)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [ ]:
'''
print(X_train.shape)
print(X_test.shape)
print(y_train)
print(y_test.shape)
3456 % len(X_train) ==0
'''
print(np.asarray(X_train).shape)
print(np.asarray(X_test).shape)
print(np.asarray(y_train).shape)
print(np.asarray(y_test).shape)

labels_train = list(map(lambda x: int(x), np.asarray(y_train)))
print(np.asarray(labels_train).shape)

In [ ]:
import pywt

scales = range(1,661)
waveletname = 'morl'
train_size = 5000
test_size= 500

#################################################
### plotting CWT of a single TEOAE signal  ######
#################################################
coeff, freq = pywt.cwt(X_train.iloc[1,:], scales, waveletname, 1)
plt.matshow(coeff,  cmap='PRGn', aspect='auto')
plt.show()
    
'''
    N = X_train.shape[0]
t0=1871
dt=0.25
time = np.arange(0, N) * dt + t0

def plot_wavelet(time, signal, scales, 
                 waveletname = 'cmor', 
                 cmap = plt.cm.seismic, 
                 title = 'Wavelet Transform (Power Spectrum) of signal', 
                 ylabel = 'Period (years)', 
                 xlabel = 'Time'):    
    dt = time[1] - time[0]
    [coefficients, frequencies] = pywt.cwt(signal, scales, waveletname, dt)
    power = (abs(coefficients)) ** 2
    period = 1. / frequencies
    levels = [0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8]
    contourlevels = np.log2(levels)
    
    fig, ax = plt.subplots(figsize=(15, 10))
    im = ax.contourf(time, np.log2(period), np.log2(power), contourlevels, extend='both',cmap=cmap)
    
    ax.set_title(title, fontsize=20)
    ax.set_ylabel(ylabel, fontsize=18)
    ax.set_xlabel(xlabel, fontsize=18)
    
    yticks = 2**np.arange(np.ceil(np.log2(period.min())), np.ceil(np.log2(period.max())))
    ax.set_yticks(np.log2(yticks))
    ax.set_yticklabels(yticks)
    ax.invert_yaxis()
    ylim = ax.get_ylim()
    ax.set_ylim(ylim[0], -1)
    
    cbar_ax = fig.add_axes([0.95, 0.5, 0.03, 0.25])
    fig.colorbar(im, cax=cbar_ax, orientation="vertical")
    plt.show()

plot_wavelet(time, X_train.iloc[1,:], scales)
'''

In [ ]:
import pywt

scales = range(1,X_train.shape[1]+1)
waveletname = 'morl'
train_size = X_train.shape[0]
test_size= X_test.shape[0]

#########################################################
### creating a tensor to store CWT matrices of train egs#
#########################################################
train_data_cwt = np.ndarray(shape=(train_size, 660, 660))
#train_data_cwt = np.ndarray(shape=(660, 660, train_size))
### a TEOAE signal has 660 samples; 
### so, its CWT is of size 660 X 660
### train_data_cwt is a tensor with train_size no. of 660X660 matrices

for ii in range(0,train_size):
    if ii % 50 == 0: #len(X_train) == 0:
        print(ii)
    signal = X_train.iloc[ii, :]
    coeff, freq = pywt.cwt(signal, scales, waveletname, 1)
    coeff_ = coeff[:,:660]
    train_data_cwt[ii, :, :] = coeff_
    #train_data_cwt[:, :, ii] = coeff_

###########################################################
### creating a tensor to store CWT matrices of test egs ###
###########################################################
test_data_cwt = np.ndarray(shape=(test_size, 660, 660))
#test_data_cwt = np.ndarray(shape=(660, 660, test_size))
### test_data_cwt is a tensor with test_size no. of 660X660 matrices

for ii in range(0,test_size):
    if ii % len(X_test) == 0:
        print(ii)
    signal = X_test.iloc[ii, :]
    coeff, freq = pywt.cwt(signal, scales, waveletname, 1)
    coeff_ = coeff[:,:660]
    test_data_cwt[ii, :, :] = coeff_
    #test_data_cwt[:, :, ii] = coeff_

###########################################################
### convert the labels from float to int of train and test egs
###########################################################
labels_train = list(map(lambda x: int(x), np.asarray(y_train)))
labels_test = list(map(lambda x: int(x), np.asarray(y_test)))

###########################################################
### putting all the train and test CWT data             ###
###########################################################
x_train = train_data_cwt
y_train = list(labels_train[:train_size])
x_test = test_data_cwt
y_test = list(labels_test[:test_size])

In [ ]:
# save numpy array
np.save('x_train_CNN', x_train)
np.save('x_test_CNN', x_test)
y_train_np = np.asarray(y_train)
np.save('y_train_CNN', y_train_np)
y_test_np = np.asarray(y_test)
np.save('y_test_CNN', y_test_np)


In [ ]:
print(np.asarray(x_train).shape)
print(np.asarray(x_test).shape)
print(np.asarray(y_train).shape)
print(np.asarray(y_test).shape)
print(len(x_train))
print(y_train)

In [3]:
#load data
x_train = np.load('x_train_CNN.npy')
x_test = np.load('x_test_CNN.npy')
y_train_np = np.load('y_train_CNN.npy')
y_test_np = np.load('y_test_CNN.npy')

In [7]:
import keras
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 
history = History()
 
img_x = x_train.shape[1] # = no.of samples of each TEOAE signal = 660
img_y = x_train.shape[1] 
img_z = 1 #x_train.shape[2] # Z =8 = no. of train egs --> chumma trying

input_shape = (img_x, img_y, img_z)
 
num_classes = 54
batch_size = 64
#num_classes = 54
epochs = 100
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print(x_train.shape)

#x_train = np.expand_dims(x_train, axis=0)
#x_test = np.expand_dims(x_test, axis=0)

#x_train = x_train.reshape(3456,660,660,1)#len(x_train),img_x, img_y, img_z)
#x_test = x_test.reshape(864,660,660,1)#len(x_test),img_x, img_y, img_z)


#(3008, 660, 660)
#(752, 660, 660)


x_train = x_train.reshape(3008,660,660,1)#len(x_train),img_x, img_y, img_z)
x_test = x_test.reshape(752,660,660,1)#len(x_test),img_x, img_y, img_z)


print(x_train.shape)
 
y_train = keras.utils.to_categorical(y_train_np, num_classes)
y_test = keras.utils.to_categorical(y_test_np, num_classes)
 
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy']) 
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=[history])
 
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))
 

(3008, 660, 660, 1)
(3008, 660, 660, 1)
Train on 3008 samples, validate on 752 samples
Epoch 1/100
3008/3008 [==============================] - 17s 6ms/step - loss: 3.9688 - acc: 0.0209 - val_loss: 3.9618 - val_acc: 0.0186
Epoch 2/100
3008/3008 [==============================] - 16s 5ms/step - loss: 3.9354 - acc: 0.0219 - val_loss: 3.9325 - val_acc: 0.0186
Epoch 3/100
3008/3008 [==============================] - 16s 5ms/step - loss: 3.9078 - acc: 0.0223 - val_loss: 3.9108 - val_acc: 0.0146
Epoch 4/100
3008/3008 [==============================] - 16s 5ms/step - loss: 3.8824 - acc: 0.0216 - val_loss: 3.8948 - val_acc: 0.0279
Epoch 5/100
3008/3008 [==============================] - 17s 6ms/step - loss: 3.8246 - acc: 0.0372 - val_loss: 3.8094 - val_acc: 0.0253
Epoch 6/100
3008/3008 [==============================] - 16s 5ms/step - loss: 3.7360 - acc: 0.0512 - val_loss: 3.6650 - val_acc: 0.0572
Epoch 7/100
3008/3008 [==============================] - 16s 5ms/step - loss: 3.5129 - acc: 0.092

In [ ]:
a=[[1 ,2 ,3,1],[4, 5 ,1,6],[6, 7, 2,8],[12,13,1, 4],[15,16,1,7]]
a=np.asarray(a)
print(a.shape)
b.reshape(2,2,2,1)

In [ ]:
from sklearn.neural_network import MLPClassifier
clf=MLPClassifier(hidden_layer_sizes=(40,10,40),solver='sgd',alpha=.0001,tol=.01,activation='relu')
#clf=MLPClassifier(hidden_layer_sizes=(1032,129,1032),solver='sgd',alpha=.0001,tol=.01,activation='tanh')
clf.fit(X_train,y_train.values.ravel())
clf.score(X_test,y_test.values.ravel())

In [ ]:
#clf.get_params()


In [ ]:
from sklearn import svm
from sklearn.svm import SVC
clf=svm.SVC(kernel='linear')
clf.fit(X_train,y_train.values.ravel())
clf.score(X_test,y_test.values.ravel())

In [ ]:
from sklearn.linear_model import Perceptron
clf=Perceptron(tol=.01,random_state=0,max_iter=50,alpha=.00001)
clf.fit(X_train,y_train.values.ravel())
clf.score(X_test,y_test.values.ravel())

In [ ]:
########
###   %history -g -f recovered.txt
########